In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/Image-Object-Localization

/content/drive/My Drive/Image-Object-Localization


In [0]:
import numpy as np

In [0]:
ls

img/  prediction/  pytorch.py  README.md  src/


In [0]:
cd src

/content/drive/My Drive/Image-Object-Localization/src


In [0]:
from PIL import Image
import numpy as np
import pickle
import random
import pandas as pd

In [0]:
base = "data/images/"

In [0]:
data = pd.read_csv('data/images.txt', sep=" ", header=None)

In [0]:
box = pd.read_csv('data/bounding_boxes.txt', sep=" ", header=None)

In [0]:
box.columns = ["id", "x", "y", "width", "height"]

In [0]:
box.head()

,id,x,y,width,height
0,1,60.0,27.0,325.0,304.0
1,2,139.0,30.0,153.0,264.0
2,3,14.0,112.0,388.0,186.0
3,4,112.0,90.0,255.0,242.0
4,5,70.0,50.0,134.0,303.0


In [0]:
data.columns = ["id","path"]

In [0]:
data.head()

,id,path
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...


In [0]:
tot_data = pd.merge(data,box,on="id", how="outer")

In [0]:
tot_data.head()

,id,path,x,y,width,height
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,60.0,27.0,325.0,304.0
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,139.0,30.0,153.0,264.0
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,14.0,112.0,388.0,186.0
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,112.0,90.0,255.0,242.0
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,70.0,50.0,134.0,303.0


In [0]:
def Normalize(image,mean,std):
    for channel in range(3):
        image[:,:,channel]=(image[:,:,channel]-mean[channel])/std[channel]
    return image

In [0]:
import cv2 
def preprocess_image(img):
    w,h = 224, 224
  
    img = cv2.resize(img, (w,h))
    img = img/255. 
    img = Normalize(img,[0.485,0.456,0.406],[0.229,0.224,0.225])

  
    return img 

In [0]:
def image_generator(input_ids, batch_size = 32):
  
    while True:
        batch_nos = np.random.choice(a= input_ids, size = batch_size)
        
        batch_image = []
        batch_box = []
    
        for input_id in batch_nos:
            image = cv2.imread(base+tot_data.iloc[input_id-1]["path"])
            boxx = {}
            boxx[0] = box.iloc[input_id-1]['x']
            boxx[1] = box.iloc[input_id-1]['y']
            boxx[2] = box.iloc[input_id-1]['width']
            boxx[3] = box.iloc[input_id-1]['height']
            boxx = np.array(list(boxx.values()))
            #print("Input id "+ str(input_id))
      
            image = preprocess_image(image)
      
            batch_image += [image]
            batch_box += [boxx]
   
        batch_ip = np.array(batch_image)
        batch_b = np.array(batch_box)
    
        yield (batch_ip, batch_b)

In [0]:
from random import shuffle

In [0]:
batch_size = 32

In [0]:
img_ids = list(tot_data["id"])

In [0]:
b[0]

array([ 69., 103., 396., 219.])

In [0]:
import tensorflow as tf
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, MaxPooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
def plot_model(model_details):
    fig, axs = plt.subplots(1,2,figsize=(15,5))

    axs[0].plot(range(1,len(model_details.history['my_metric'])+1),model_details.history['my_metric'])
    axs[0].plot(range(1,len(model_details.history['val_my_metric'])+1),[1.7*x for x in model_details.history['val_my_metric']])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_details.history['my_metric'])+1),len(model_details.history['my_metric'])/10)
    axs[0].legend(['train', 'val'], loc='best')

    axs[1].plot(range(1,len(model_details.history['loss'])+1),model_details.history['loss'])
    axs[1].plot(range(1,len(model_details.history['val_loss'])+1),model_details.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_details.history['loss'])+1),len(model_details.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')

    plt.savefig("model.png")

In [0]:
def my_metric(labels,predictions):
    threshhold=0.75
    x=predictions[:,0]*224
    x=tf.maximum(tf.minimum(x,224.0),0.0)
    y=predictions[:,1]*224
    y=tf.maximum(tf.minimum(y,224.0),0.0)
    width=predictions[:,2]*224
    width=tf.maximum(tf.minimum(width,224.0),0.0)
    height=predictions[:,3]*224
    height=tf.maximum(tf.minimum(height,224.0),0.0)
    label_x=labels[:,0]
    label_y=labels[:,1]
    label_width=labels[:,2]
    label_height=labels[:,3]
    a1=tf.multiply(width,height)
    a2=tf.multiply(label_width,label_height)
    x1=tf.maximum(x,label_x)
    y1=tf.maximum(y,label_y)
    x2=tf.minimum(x+width,label_x+label_width)
    y2=tf.minimum(y+height,label_y+label_height)
    IoU=tf.abs(tf.multiply((x1-x2),(y1-y2)))/(a1+a2-tf.abs(tf.multiply((x1-x2),(y1-y2))))
    condition=tf.less(threshhold,IoU)
    sum=tf.where(condition,tf.ones(tf.shape(condition)),tf.zeros(tf.shape(condition)))
    return tf.reduce_mean(sum)

In [0]:
def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

In [0]:
def resnet_block(inputs,num_filters,kernel_size,strides,activation='relu'):
    x=Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(inputs)
    x=BatchNormalization()(x)
    if(activation):
        x=Activation('relu')(x)
    return x

In [0]:
def resnet18():
    inputs=Input((224,224,3))
    
    # conv1
    x=resnet_block(inputs,64,[7,7],2)

    # conv2
    x=MaxPooling2D([3,3],2,'same')(x)
    for i in range(2):
        a=resnet_block(x,64,[3,3],1)
        b=resnet_block(a,64,[3,3],1,activation=None)
        x=keras.layers.add([x,b])
        x=Activation('relu')(x)
    
    # conv3
    a=resnet_block(x,128,[1,1],2)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=Conv2D(128,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,128,[3,3],1)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv4
    a=resnet_block(x,256,[1,1],2)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=Conv2D(256,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,256,[3,3],1)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv5
    a=resnet_block(x,512,[1,1],2)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=Conv2D(512,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,512,[3,3],1)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    x=AveragePooling2D(pool_size=7,data_format="channels_last")(x)
    # out:1*1*512

    y=Flatten()(x)
    # out:512
    y=Dense(1000,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    outputs=Dense(4,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    
    model=Model(inputs=inputs,outputs=outputs)
    return model

In [0]:
model = resnet18()
model.compile(loss=smooth_l1_loss,optimizer=Adam(),metrics=[my_metric])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [0]:
def lr_sch(epoch):
    #200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5

In [0]:
lr_scheduler=LearningRateScheduler(lr_sch)
lr_reducer=ReduceLROnPlateau(monitor='val_my_metric',factor=0.2,patience=5,mode='max',min_lr=1e-3)

In [0]:
train_steps = 150
valid_steps = 50

In [0]:
checkpoint=ModelCheckpoint('model.h5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
model_details=model.fit_generator(train_generator,validation_data = valid_generator,epochs=10,steps_per_epoch = train_steps, validation_steps = valid_steps,callbacks=[lr_scheduler,lr_reducer,checkpoint],verbose=1)
model.save('model.h5')

Epoch 1/10
150/150 [==============================] - 1428s 10s/step - loss: 752.5543 - my_metric: 0.0090 - val_loss: 326.8605 - val_my_metric: 0.0013
Epoch 2/10
150/150 [==============================] - 845s 6s/step - loss: 286.2998 - my_metric: 0.0031 - val_loss: 283.9760 - val_my_metric: 0.0019
Epoch 3/10
150/150 [==============================] - 489s 3s/step - loss: 245.9402 - my_metric: 0.0069 - val_loss: 321.5000 - val_my_metric: 6.2500e-04
Epoch 4/10
150/150 [==============================] - 372s 2s/step - loss: 234.1981 - my_metric: 0.0112 - val_loss: 241.2444 - val_my_metric: 0.0063
Epoch 5/10
150/150 [==============================] - 216s 1s/step - loss: 212.1703 - my_metric: 0.0169 - val_loss: 356.7574 - val_my_metric: 6.2500e-04
Epoch 6/10
150/150 [==============================] - 156s 1s/step - loss: 200.4717 - my_metric: 0.0142 - val_loss: 217.8022 - val_my_metric: 0.0181
Epoch 7/10
150/150 [==============================] - 131s 871ms/step - loss: 188.2325 - my_metr

In [0]:
#scores=model.evaluate(data_test,box_test,verbose=1)
scores = model.evaluate_generator(valid_generator, steps = 50, workers=1, verbose=1)
print('Test loss : ',scores[0])
print('Test accuracy : ',scores[1])

#plot_model(model_details)

50/50 [==============================] - 19s 380ms/step
Test loss :  230.67255279541016
Test accuracy :  0.025625


NameError: ignored

In [0]:
ans = model.predict_generator(valid_generator, steps=100, max_queue_size=10, workers=1, verbose=1)

100/100 [==============================] - 39s 386ms/step


In [0]:
ans

array([[0.5215432 , 0.55674475, 1.0110135 , 0.8146289 ],
       [0.7627302 , 0.51549363, 0.783108  , 0.7985282 ],
       [0.51536506, 0.22136112, 1.1119565 , 1.5042298 ],
       ...,
       [0.90562886, 0.33056402, 1.1191331 , 1.0847689 ],
       [0.76997906, 0.47528508, 0.88231385, 1.0414791 ],
       [0.5021735 , 0.2558036 , 1.025503  , 1.304435  ]], dtype=float32)

In [0]:
ind = 500
ig_path = tot_data.iloc[ind-1]["path"]
ig_path
ig_dat = Image.open(base+ig_path).convert('RGB')

In [0]:
ig_dat=ig_dat.resize((224,224))
ig_dat=np.array(ig_dat,dtype=np.float32)
ig_dat=ig_dat/255
ig_dat=Normalize(ig_dat,[0.485,0.456,0.406],[0.229,0.224,0.225])

In [0]:
boxx = {}
boxx[0] = box.iloc[ind-1]['x']
boxx[1] = box.iloc[ind-1]['y']
boxx[2] = box.iloc[ind-1]['width']
boxx[3] = box.iloc[ind-1]['height']
boxx = np.array(list(boxx.values()))

In [0]:
result=model.predict(ig_dat[:,:,:])

ValueError: ignored